In [ ]:
%cd facenet_v2/src
!pwd
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from scipy import misc
from PIL import Image

In [ ]:
import tensorflow as tf
import numpy as np
import sys
import os
import copy
import argparse
import facenet
import align.detect_face
import imageio

In [ ]:
def main(lis_name,model_path,lis_em):

    images = load_data(lis_name, 160, 1)
    
    with tf.Graph().as_default():

        with tf.compat.v1.Session() as sess:
      
            # Load the model
            facenet.load_model(model_path)
    
            # Get input and output tensors
            images_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.compat.v1.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")
            feed_dict = { images_placeholder: images, phase_train_placeholder:False }
            emb = sess.run(embeddings, feed_dict=feed_dict)
            lis_em.append(emb)
            nrof_images = len(lis_name)
    return lis_em             
            
def load_data(image_paths, image_size, gpu_memory_fraction):
    img_list = []
    for image in image_paths:
        img = np.array(Image.open(image)).astype(np.float32)
        prewhitened = (img - 127.5)/128
        img_list.append(prewhitened)
    images = np.stack(img_list)
    return images

In [ ]:
lis_em=[]
count = 0
lis_name=[]
classMap=dict()
lis_name_las=[]
#model_path='./evaluate'
model_path='../../facenet_triplet/models/triplet/20221018-221109'
from glob import glob
len_image=len([name for name in glob('../../new-cluster_160/*/*.png') if os.path.isfile(name)])
print(len_image)

In [ ]:
for f in glob('../../new-cluster_160/*/*.png'):
            if count % 100 == 0:
                print(f'#{count}')
                if len(lis_name)!=0 and count<len_image:
                    print(lis_name)
                    print(len(lis_name))
                    main(lis_name,model_path,lis_em)
                    lis_name=[]
                    print('len_em:',len(lis_em))
                    print('--------------------------------------')                    
            
            paths = f.split('/')
            cls = paths[-2]
            name = paths[-1]
            classMap[name]=cls           
            lis_name.append(f)
            lis_name_las.append(name)
            
            count += 1
            
            if (count==len_image) and (len(lis_name)!=0):
                    main(lis_name,model_path,lis_em)
                    print(lis_name)
                    lis_name=[]
                    print('len_em:',len(lis_em))
print(count)

In [ ]:
#lưu embedding và name
import pickle
with open("./kde/my_embed1", "wb") as f_em:
    pickle.dump(lis_em, f_em)
with open("./kde/my_name1", "wb") as f_name:   
    pickle.dump(lis_name_las,f_name)
with open("./kde/class_map1", "wb") as f_class:   
    pickle.dump(classMap,f_class)
load
import pickle
with open("./kde/my_embed1", "rb") as f_em: 
    lis_em = pickle.load(f_em)
with open("./kde/my_name1", "rb") as f_name:  
    lis_name_las = pickle.load(f_name)
with open("./kde/class_map1", "rb") as f_class:   
    classMap=pickle.load(f_class)

In [ ]:
#xử lí embedding
mang=[]
for x in lis_em:
    for i in x:
        mang.append(i)
my_embed=np.array([mang]).squeeze(0)
np.save('./kde/embed_arr1.npy', my_embed)
print(len(mang))
print(lis_name_las[0])
print(classMap[lis_name_las[0]])

In [ ]:
my_embed=np.load('./kde/embed_arr1.npy', allow_pickle = True)
print(my_embed.shape)

In [ ]:
from scipy.spatial import distance
dis1 = distance.cdist(my_embed, my_embed, 'euclidean')
np.save('./kde/distance1.npy', dis1)
matrix=[]
print(dis1.shape[0],dis1.shape[1])
for i in range(1,dis1.shape[0]):
    for j in range(0,i):
        classify = 'TRUE' if classMap[lis_name_las[i]]==classMap[lis_name_las[j]] else 'FALSE'
        matrix.append([lis_name_las[i],lis_name_las[j],dis1[i,j],classify])
print(len(matrix))

In [ ]:
#plot kde
dist=np.array([float(row[2]) for row in matrix])
label = np.array([row[3] == 'TRUE' for row in matrix])
thresholds = np.arange(np.min(dist), np.max(dist), 0.01)
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
density_yes = gaussian_kde(dist[label])
density_no = gaussian_kde(dist[label == False])
plt.plot(thresholds, density_yes(thresholds), label='yes')
plt.plot(thresholds, density_no(thresholds), label='no')
plt.legend()
plt.xlabel('distance')
plt.show(

In [ ]:
import seaborn
seaborn.kdeplot(data=dist[label])
seaborn.kdeplot(data=dist[label == False])